In [5]:
import energyusage

In [11]:
# user function to be evaluated
def recursive_fib():
    clf = AdaBoostClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)
    print("Accuracy:", accuracy)

energyusage.evaluate(recursive_fib, 40, pdf=True)

Location:                                                                Florida


PermissionError: [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl:0:2/energy_uj'

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.svm import SVC
from math import sqrt
import time
import tracemalloc
import xgboost as xgb

In [2]:
df=pd.read_csv('GrammarandProductReviews[modified].csv')

In [3]:
cv = CountVectorizer()
cv.fit(df['review'])
X = cv.transform(df['review'])

In [4]:
LogReg_clf = LogisticRegression()
SVC_clf = SVC(probability=True, kernel='rbf')
NV=MultinomialNB()
NK=BernoulliNB()
DTree_clf = DecisionTreeClassifier()
svm=LinearSVC()
rf = RandomForestClassifier(random_state=1200, criterion='entropy', n_estimators=200)
#ada = AdaBoostClassifier(n_estimators=200, learning_rate=2.0, algorithm='SAMME')
ada = AdaBoostClassifier()
ETree = ExtraTreesClassifier(random_state=0)

In [5]:
msg_train, msg_test, label_train, label_test =train_test_split(X, df['positive_review'], test_size=0.2,random_state=42)

In [7]:
# get the start time
st = time.time()
tracemalloc.start()


NV.fit(msg_train,label_train)
preds= NV.predict(msg_test)
acc = accuracy_score(label_test, preds)
print("Accuracy For NB Model is: " + str(acc))
rms= sqrt(mean_squared_error(label_test, preds))
print("RMSE Error is: " + str(rms))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
#print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy For NB Model is: 0.8979400749063671
RMSE Error is: 0.31946819105136737
Execution time: 0.019340991973876953 seconds
Current memory usage is 0.37818145751953125 MB; Peak was 0.8618383407592773 MB


In [7]:
#Voting Ensemble Method
# get the start time
st = time.time()
tracemalloc.start()



#voting_clf = VotingClassifier(estimators=[('LR',model),('NV', ada)], voting='hard')
voting_clf = VotingClassifier(estimators=[('svm',SVC_clf ), ('AdaBoost', ada), ('RF',rf )], voting='soft')
voting_clf.fit(msg_train, label_train)
preds = voting_clf.predict(msg_test)

acc = accuracy_score(label_test, preds)
rms= sqrt(mean_squared_error(label_test, preds))


print("Accuracy For Voting Ensemble SVM+AadBoost+RF is: " + str(acc))
print("RMSE Error is: " + str(rms))


# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy For Voting Ensemble SVM+AadBoost+RF is: 0.9161985018726592
RMSE Error is: 0.28948488410855033
Execution time: 64.94330596923828 seconds
Current memory usage is 1.525527000427246 MB; Peak was 11.562824249267578 MB


In [8]:
# Boosting Ensemble (Adaboost classifier)

# get the start time
st = time.time()
tracemalloc.start()

clf = AdaBoostClassifier()
clf.fit(msg_train, label_train)

# make predictions on the testing set
y_pred = clf.predict(msg_test)

# evaluate the model's performance using accuracy and RMSE
accuracy = accuracy_score(label_test, y_pred)
rmse = mean_squared_error(label_test, y_pred)

print("Accuracy for AdaBoost:", accuracy)
print("RMSE:", rmse)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy for AdaBoost: 0.87312734082397
RMSE: 0.12687265917602997
Execution time: 2.937202215194702 seconds
Current memory usage is 0.06054115295410156 MB; Peak was 3.302509307861328 MB


In [9]:
# Train XGBoost model
# get the start time
st = time.time()
tracemalloc.start()


model = xgb.XGBClassifier(n_estimators=70, learning_rate=0.9)
model.fit(msg_train, label_train)

# Predict on test set and calculate accuracy
y_pred = model.predict(msg_test)
acc = accuracy_score(label_test, y_pred)
print('Accuracy for XGBoost:', acc)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

current, peak = tracemalloc.get_traced_memory()
# Stop tracemalloc
tracemalloc.stop()
#print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")
print("Current memory usage is", current / (1024 * 1024), "MB; Peak was", peak / (1024 * 1024), "MB")

Accuracy for XGBoost: 0.9194756554307116
Execution time: 0.6898694038391113 seconds
Current memory usage is 0.10698986053466797 MB; Peak was 0.18793773651123047 MB


In [6]:
import energyusage
# user function to be evaluated
def recursive_fib():
        voting_clf = VotingClassifier(estimators=[('svm',SVC_clf ), ('AdaBoost', ada), ('RF',rf )], voting='soft')
        voting_clf.fit(msg_train, label_train)
        preds = voting_clf.predict(msg_test)

        acc = accuracy_score(label_test, preds)
        rms= sqrt(mean_squared_error(label_test, preds))


        print("Accuracy For Voting Ensemble SVM+AadBoost+RF is: " + str(acc))
        print("RMSE Error is: " + str(rms))

energyusage.evaluate(recursive_fib, 40, pdf=True)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Location:                                                                Florida


PermissionError: [Errno 13] Permission denied: '/sys/class/powercap/intel-rapl:0:2/energy_uj'